In [1]:
import numpy as np


In [2]:

input = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
result=15

# input = [[10, 11, 10, 11], [2, 21, 20, 10], [1, 20, 21, 11], [2, 1, 2, 1]]
# height = 1
# result=18

n = len(input)
npinput = np.asarray(input)

In [3]:
# group api
groups = []

# gi=group index. -1=new
# return ; group index
def group_add(i,j, gi):
    global groups
    if gi==-1:
        groups.append([(i,j)])
        gi = len(groups)-1
        print('newgroup:',gi, 'item=', i,j)
    else:
        groups[gi].append((i,j))
        print('add group=',gi, 'item=', i,j)
    return gi

def group_find(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    for gi, g in enumerate(groups):
        for f in g:
            if f[0]==i and f[1]==j:
                return gi
    return -1

# d = l, r, u, d  (left, right, up, down)
def can_move(i,j, d):
    i2,j2 = i,j
    if d=='l': j2 -=1
    if d=='r': j2 +=1
    if d=='u': i2 -=1
    if d=='d': i2 +=1
    if i2<0 or j2<0:
        return False
    if i2>=n or j2>=n:
        return False
    return abs(input[i][j] - input[i2][j2])<=height

def group_merge(gi, gj):
    global groups
    bak = groups[gj][:]
    print('group merge ',gj,'into',gi)
    groups[gi].extend(bak)
    del groups[gj]
    

In [4]:
groups.clear()
print(group_add(0,0,-1))
print(group_add(1,1,0))
print(group_add(2,2,-1))
print(group_add(3,3,1))
print(group_add(4,4,-1))
print(group_add(5,6,2))

print('find=', group_find(5,6))
print('find=', group_find(2,2))

groups

newgroup: 0 item= 0 0
0
add group= 0 item= 1 1
0
newgroup: 1 item= 2 2
1
add group= 1 item= 3 3
1
newgroup: 2 item= 4 4
2
add group= 2 item= 5 6
2
find= -1
find= 1


[[(0, 0), (1, 1)], [(2, 2), (3, 3)], [(4, 4), (5, 6)]]

In [5]:
groups

[[(0, 0), (1, 1)], [(2, 2), (3, 3)], [(4, 4), (5, 6)]]

In [6]:
group_merge(1,2)

group merge  2 into 1


In [7]:
groups

[[(0, 0), (1, 1)], [(2, 2), (3, 3), (4, 4), (5, 6)]]

In [8]:
can_move(0,2,'r')

True

In [9]:
global groups
print(input)
groups.clear()

for i in range(n):
    for j in range(n):
        gi = group_find(i,j)
        if gi==-1:
            gi = group_add(i,j, -1)
        print('check ', i,j, 'group=', gi)
            
        if can_move(i,j,'r'):
            g2 = group_find(i,j+1)
            if g2==-1:
                group_add(i,j+1,gi)
            elif g2==gi:
                pass
            else:
                group_merge(min(gi,g2), max(gi,g2))
                gi = min(gi, g2)

        if can_move(i,j,'d'):
            g2 = group_find(i+1,j)
            if g2==-1:
                group_add(i+1,j,gi)
            elif g2==gi:
                pass
            else:
                group_merge(min(gi,g2), max(gi,g2))

print(groups)


[[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
newgroup: 0 item= 0 0
check  0 0 group= 0
add group= 0 item= 0 1
check  0 1 group= 0
add group= 0 item= 1 1
newgroup: 1 item= 0 2
check  0 2 group= 1
add group= 1 item= 0 3
add group= 1 item= 1 2
check  0 3 group= 1
newgroup: 2 item= 1 0
check  1 0 group= 2
group merge  2 into 0
check  1 1 group= 0
group merge  1 into 0
check  1 2 group= 0
add group= 0 item= 1 3
check  1 3 group= 0
newgroup: 1 item= 2 0
check  2 0 group= 1
add group= 1 item= 2 1
add group= 1 item= 3 0
check  2 1 group= 1
add group= 1 item= 2 2
add group= 1 item= 3 1
check  2 2 group= 1
add group= 1 item= 2 3
add group= 1 item= 3 2
check  2 3 group= 1
check  3 0 group= 1
check  3 1 group= 1
check  3 2 group= 1
newgroup: 2 item= 3 3
check  3 3 group= 2
[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3)], [(2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2)], [(3, 3)]]


In [10]:
groups

[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3)],
 [(2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2)],
 [(3, 3)]]

In [11]:
# 가장 짧은 곳을 연결. merge
bridge=[] # (i1,j1,i2,j2,cost)
while len(groups)>1:
    firstgroup = groups[0]
    tmpb = [] 
    mincost=99999
    mincostidx=0
    mincostgroup=0
    for it in firstgroup:
        cost=-1
        # 다른 그룹과 접하는지 검사. lrud
        # 접한다면 비용 기록. (누구랑, 누구랑 연결할 때 비용)
        gi = group_find(it[0], it[1]-1)
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]-1])
            tmpb.append( (it[0], it[1], it[0], it[1]-1, cost) )
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0], it[1]+1)
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]+1])
            tmpb.append( (it[0], it[1], it[0], it[1]+1, cost) )           
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0]-1, it[1])
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]-1][it[1]])
            tmpb.append( (it[0], it[1], it[0]-1, it[1], cost) ) 
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0]+1, it[1])
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]+1][it[1]])
            tmpb.append( (it[0], it[1], it[0]+1, it[1], cost) )           
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
    print(tmpb)
    print('cost=',mincost, 'idx=',mincostidx, 'group=', mincostgroup)
    # 최소 비용 캐치. 다리 건설.
    bridge.append(tmpb[mincostidx])
    # 1그룹으로 병합
    group_merge(0, mincostgroup)

print(groups)        
print(bridge)

# total cost
total = 0
for x in bridge:
    total += x[4]
print(total)

[(1, 1, 2, 1, 5), (1, 0, 2, 0, 5), (1, 2, 2, 2, 5), (1, 3, 2, 3, 5)]
cost= 5 idx= 0 group= 1
group merge  1 into 0
[(2, 3, 3, 3, 10), (3, 2, 3, 3, 10)]
cost= 10 idx= 0 group= 1
group merge  1 into 0
[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3), (2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2), (3, 3)]]
[(1, 1, 2, 1, 5), (2, 3, 3, 3, 10)]
15


In [12]:
input=[]
n = 0
groups=[]
gheight=0
bdebug=True


# gi=group index. -1=new
# return ; group index
def group_add(i,j, gi):
    global groups
    if gi==-1:
        groups.append([(i,j)])
        gi = len(groups)-1
        if bdebug:
            print('newgroup:',gi, 'item=', i,j)
    else:
        groups[gi].append((i,j))
        if bdebug:
            print('add group=',gi, 'item=', i,j)
    return gi

def group_find(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    for gi, g in enumerate(groups):
        for f in g:
            if f[0]==i and f[1]==j:
                return gi
    return -1

# d = l, r, u, d  (left, right, up, down)
def can_move(i,j, d):
    i2,j2 = i,j
    if d=='l': j2 -=1
    if d=='r': j2 +=1
    if d=='u': i2 -=1
    if d=='d': i2 +=1
    if i2<0 or j2<0:
        return False
    if i2>=n or j2>=n:
        return False
    return abs(input[i][j] - input[i2][j2])<=gheight

def group_merge(gi, gj):
    global groups
    bak = groups[gj][:]
    if bdebug:
        print('group merge ',gj,'into',gi)
    groups[gi].extend(bak)
    del groups[gj]
    
    
def solution(land, height):
    global input, n, groups, gheight
    
    input=land
    gheight = height
    n=len(input)

    print("N=", n)

    for i in range(n):
        if bdebug:
            print("progress ", i, "/", n)
            
        for j in range(n):
            gi = group_find(i,j)
            if gi==-1:
                gi = group_add(i,j, -1)

            if can_move(i,j,'r'):
                g2 = group_find(i,j+1)
                if g2==-1:
                    group_add(i,j+1,gi)
                elif g2==gi:
                    pass
                else:
                    group_merge(min(gi,g2), max(gi,g2))
                    gi = min(gi, g2)

            if can_move(i,j,'d'):
                g2 = group_find(i+1,j)
                if g2==-1:
                    group_add(i+1,j,gi)
                elif g2==gi:
                    pass
                else:
                    group_merge(min(gi,g2), max(gi,g2))
    if bdebug:
        print(groups)
        print('group count=', len(groups))
        
    return 0 ;

    # 가장 짧은 곳을 연결. merge
    bridge=[] # (i1,j1,i2,j2,cost)
    while len(groups)>1:
        firstgroup = groups[0]
        tmpb = [] 
        mincost=99999
        mincostidx=0
        mincostgroup=0
        for it in firstgroup:
            cost=-1
            # 다른 그룹과 접하는지 검사. lrud
            # 접한다면 비용 기록. (누구랑, 누구랑 연결할 때 비용)
            gi = group_find(it[0], it[1]-1)
            if gi>0:
                cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]-1])
                tmpb.append( (it[0], it[1], it[0], it[1]-1, cost) )
                if mincost>cost:
                    mincost=cost
                    mincostidx=len(tmpb)-1
                    mincostgroup=gi
            gi = group_find(it[0], it[1]+1)
            if gi>0:
                cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]+1])
                tmpb.append( (it[0], it[1], it[0], it[1]+1, cost) )           
                if mincost>cost:
                    mincost=cost
                    mincostidx=len(tmpb)-1
                    mincostgroup=gi
            gi = group_find(it[0]-1, it[1])
            if gi>0:
                cost = abs(input[it[0]][it[1]]-input[it[0]-1][it[1]])
                tmpb.append( (it[0], it[1], it[0]-1, it[1], cost) ) 
                if mincost>cost:
                    mincost=cost
                    mincostidx=len(tmpb)-1
                    mincostgroup=gi
            gi = group_find(it[0]+1, it[1])
            if gi>0:
                cost = abs(input[it[0]][it[1]]-input[it[0]+1][it[1]])
                tmpb.append( (it[0], it[1], it[0]+1, it[1], cost) )           
                if mincost>cost:
                    mincost=cost
                    mincostidx=len(tmpb)-1
                    mincostgroup=gi
        if bdebug:
            print('cost=',mincost, 'idx=',mincostidx, 'group=', mincostgroup)
        # 최소 비용 캐치. 다리 건설.
        bridge.append(tmpb[mincostidx])
        # 1그룹으로 병합
        group_merge(0, mincostgroup)

    if bdebug:
        print(groups)        
    if bdebug:
        print(bridge)

    # total cost
    total = 0
    for x in bridge:
        total += x[4]
    if bdebug:
        print('total=', total)
    answer = total
    return answer

In [13]:
import numpy as np


In [ ]:
# stress test
npinput = np.random.rand(300,300)*100
npinput = np.asarray(npinput, dtype=int)
print(npinput)
solution(npinput.tolist(), 50)

In [ ]:

input = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
result=15

# input = [[10, 11, 10, 11], [2, 21, 20, 10], [1, 20, 21, 11], [2, 1, 2, 1]]
# height = 1
# result=18

solution(input, 3)

In [17]:
np.abs(npinput[0]-npinput[0][0])

array([ 0, 15,  1, 47, 23, 15, 33, 61, 77, 15, 53, 24, 41, 13, 14, 77, 16,
        5, 14, 23, 42,  3, 79,  5, 16,  3, 74, 69, 60, 60, 41,  1, 61, 65,
       37, 56, 15, 40, 15, 17, 10, 38,  5, 66, 19, 16,  8, 22, 44, 35, 21,
       68, 35, 18, 38, 65, 73, 38, 14, 14,  9, 10, 54,  5,  5, 16,  4, 77,
       15, 17, 79, 17, 46, 61, 41, 61, 20, 17, 12, 51, 28, 73, 76,  9, 77,
       57, 36,  3, 65, 27, 37, 15, 41, 76,  7, 26, 58, 38, 78, 59, 60, 13,
       19, 16, 48, 44, 13, 24, 11, 26, 20,  9, 10, 69, 61, 22,  1, 61, 18,
        7,  8, 22, 20,  6, 49, 14, 38, 45, 75, 25,  6, 36, 62,  5, 11, 16,
       36, 33, 23, 36,  3, 45, 16, 19, 67, 46, 63, 50, 20, 54,  2, 46, 48,
       20,  3, 28, 56, 36, 29, 31, 51, 73, 50,  6, 28, 63, 19,  3, 40, 59,
       26, 77, 15, 52, 61, 12, 71, 20, 70, 78, 43, 63, 19, 14, 67, 25, 32,
       77, 76, 20, 68, 14, 13, 21,  8, 70, 60, 70,  4, 70, 51, 12, 17, 19,
       71, 19,  6, 70, 28,  5, 69,  6,  1,  9, 62,  4, 53, 35,  0, 42,  3,
       12, 17, 66, 59, 53